# Game Bug Classifier - Complete Pipeline
## Label Improvement → Training → Evaluation

**Requirements:**
- Kaggle GPU enabled
- Internet enabled
- Input dataset: `data-splits` (train.jsonl, val.jsonl, test.jsonl)

**Runtime:** ~5-6 hours total
- Label improvement: ~3 hours
- Training: ~2 hours  
- Evaluation: ~5 minutes

In [ ]:
# Install required packages
!pip install -q transformers==4.46.3 peft==0.13.2 bitsandbytes==0.45.0 accelerate==1.2.1

In [ ]:
# Verify paths
!ls /kaggle/input/data-splits/
!echo "\nWorking directory:"
!pwd

## Step 1: Label Improvement with Mistral Base Model

Uses the instruction-tuned Mistral-7B base model to generate better labels than keyword-based heuristics.

In [ ]:
import os
import json
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from tqdm import tqdm

def load_dataset(path):
    """Load JSONL dataset"""
    data = []
    with open(path, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    return data

def improve_labels_with_mistral(bug_report, model, tokenizer):
    """Generate improved labels using Mistral base model"""
    instruction = """Classify this game engine bug report. Provide labels in exactly this format:
Severity: [critical/major/moderate/minor]
Component: [rendering/physics/ui/networking/audio/gameplay/other]
Reproducibility: [always/often/sometimes/rare]

Guidelines:
- critical: crashes, data loss, security, game-breaking
- major: broken features, significant gameplay impact
- moderate: noticeable issues with workarounds
- minor: cosmetic, polish, minor annoyances

- always: 100% reproduction
- often: >70% reproduction  
- sometimes: 30-70% reproduction
- rare: <30% reproduction or unclear steps

Base classification on context and impact, not just keywords."""
    
    prompt = f"""### Instruction:
{instruction}

### Input:
{bug_report}

### Output:
"""
    
    try:
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024)
        inputs = {k: v.to(model.device) for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=80,
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id
            )
        
        generated = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
        return generated.strip()
    
    except Exception as e:
        print(f"Error: {e}")
        return None

# Configuration
MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.2"
TRAIN_PATH = "/kaggle/input/data-splits/train.jsonl"
OUTPUT_PATH = "/kaggle/working/train_improved.jsonl"

print("="*70)
print("🔬 LABEL QUALITY IMPROVEMENT")
print("="*70)

# Load training data
print(f"\n📂 Loading training data...")
train_data = load_dataset(TRAIN_PATH)
print(f"✅ Loaded {len(train_data)} examples")

# Load Mistral base model
print(f"\n🔍 Loading Mistral-7B-Instruct base model...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
model.eval()
print("✅ Model loaded")

# Improve labels
print(f"\n🚀 Generating improved labels...")
improved_data = []
failed = 0

for example in tqdm(train_data, desc="Improving labels"):
    improved_output = improve_labels_with_mistral(example['input'], model, tokenizer)
    
    if improved_output:
        improved_example = {
            'instruction': example['instruction'],
            'input': example['input'],
            'output': improved_output,
            'metadata': {
                'original_output': example['output'],
                'improvement_method': 'mistral-base-model'
            }
        }
        improved_data.append(improved_example)
    else:
        failed += 1
        improved_data.append(example)

# Save improved dataset
print(f"\n💾 Saving improved dataset...")
with open(OUTPUT_PATH, 'w') as f:
    for example in improved_data:
        f.write(json.dumps(example) + '\n')

print(f"✅ Saved {len(improved_data)} examples")
print(f"⚠️  Failed: {failed} examples")

# Show comparison samples
print("\n" + "="*70)
print("📊 SAMPLE COMPARISON (First 2)")
print("="*70)
for i in range(min(2, len(improved_data))):
    print(f"\n--- Example {i+1} ---")
    print(f"Input: {improved_data[i]['input'][:100]}...")
    print(f"\nOriginal: {improved_data[i]['metadata']['original_output']}")
    print(f"\nImproved: {improved_data[i]['output']}")
    print("-" * 70)

print("\n✨ Label improvement complete!")

# Clean up to free memory
del model
del tokenizer
torch.cuda.empty_cache()

## Step 2: Train Model with Improved Labels

Fine-tune Mistral-7B with QLoRA on the improved labels.

In [ ]:
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Configuration
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"
TRAIN_PATH = "/kaggle/working/train_improved.jsonl"
VAL_PATH = "/kaggle/input/data-splits/val.jsonl"
OUTPUT_DIR = "/kaggle/working/outputs"

print("="*60)
print("🎮 TRAINING")
print("="*60)

# Load data
print("\n📂 Loading datasets...")
train_data = load_dataset(TRAIN_PATH)
val_data = load_dataset(VAL_PATH)
train_dataset = Dataset.from_list(train_data)
val_dataset = Dataset.from_list(val_data)
print(f"Train: {len(train_dataset)} | Val: {len(val_dataset)}")

# Load tokenizer
print("\n🤖 Loading model...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load model with 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

model.config.use_cache = False
model.config.pretraining_tp = 1
model = prepare_model_for_kbit_training(model)

# Setup LoRA
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"]
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# Tokenize function
def format_and_tokenize(example):
    prompt = f"""### Instruction:
{example['instruction']}

### Input:
{example['input']}

### Output:
{example['output']}"""
    
    tokenized = tokenizer(
        prompt,
        truncation=True,
        max_length=512,
        padding='max_length',
    )
    tokenized['labels'] = tokenized['input_ids'].copy()
    return tokenized

# Tokenize datasets
print("\n🔤 Tokenizing...")
train_dataset = train_dataset.map(
    format_and_tokenize,
    remove_columns=train_dataset.column_names,
    desc="Tokenizing training data"
)

val_dataset = val_dataset.map(
    format_and_tokenize,
    remove_columns=val_dataset.column_names,
    desc="Tokenizing validation data"
)

# Training arguments
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    save_strategy="steps",
    save_steps=200,
    eval_strategy="steps",
    eval_steps=200,
    logging_steps=50,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    push_to_hub=False,
    report_to=["tensorboard"],
    optim="paged_adamw_32bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
)

# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
)

# Train
print("\n🚀 Starting training...")
trainer.train()

# Save final model
print("\n💾 Saving final model...")
final_output_dir = os.path.join(OUTPUT_DIR, "final_model")
trainer.save_model(final_output_dir)
tokenizer.save_pretrained(final_output_dir)

print(f"\n✅ Training complete! Model saved to: {final_output_dir}")

## Step 3: Evaluate Model

Test the fine-tuned model on held-out test set.

In [ ]:
from peft import PeftModel

# Configuration
MODEL_PATH = "/kaggle/working/outputs/final_model"
BASE_MODEL = "mistralai/Mistral-7B-Instruct-v0.2"
TEST_PATH = "/kaggle/input/data-splits/test.jsonl"
NUM_SAMPLES = 50

print("="*70)
print("🎯 EVALUATION")
print("="*70)

# Load test data
print(f"\n📂 Loading test data...")
test_data = load_dataset(TEST_PATH)
test_data = test_data[:NUM_SAMPLES]
print(f"✅ Testing on {len(test_data)} examples")

# Load model
print(f"\n🔍 Loading model...")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

model = PeftModel.from_pretrained(base_model, MODEL_PATH)
model.eval()
print("✅ Model loaded")

# Generate predictions
def generate_prediction(model, tokenizer, instruction, input_text):
    prompt = f"""### Instruction:
{instruction}

### Input:
{input_text}

### Output:
"""
    
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    generated = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    return generated.strip()

def parse_output(text):
    fields = {}
    for field in ['severity', 'component', 'reproducibility']:
        key = field.capitalize() + ":"
        if key in text:
            start = text.index(key) + len(key)
            end = text.find("\n", start)
            if end == -1:
                end = len(text)
            fields[field] = text[start:end].strip().lower()
    return fields

print("\n🚀 Generating predictions...")
predictions = []
ground_truths = []

for example in tqdm(test_data, desc="Evaluating"):
    pred = generate_prediction(model, tokenizer, example['instruction'], example['input'])
    predictions.append(pred)
    ground_truths.append(example['output'])

# Calculate accuracy
def calculate_accuracy(predictions, ground_truths, field):
    correct = 0
    total = 0
    
    for pred, gt in zip(predictions, ground_truths):
        pred_fields = parse_output(pred)
        gt_fields = parse_output(gt)
        
        if field in pred_fields and field in gt_fields:
            if pred_fields[field] == gt_fields[field]:
                correct += 1
            total += 1
    
    return (correct / total * 100) if total > 0 else 0.0

print("\n" + "="*70)
print("📊 RESULTS")
print("="*70)

severity_acc = calculate_accuracy(predictions, ground_truths, 'severity')
component_acc = calculate_accuracy(predictions, ground_truths, 'component')
repro_acc = calculate_accuracy(predictions, ground_truths, 'reproducibility')
overall = (severity_acc + component_acc + repro_acc) / 3

print(f"\nSeverity:       {severity_acc:.2f}%")
print(f"Component:      {component_acc:.2f}%")
print(f"Reproducibility: {repro_acc:.2f}%")
print(f"\n{'='*70}")
print(f"Overall:        {overall:.2f}%")
print(f"{'='*70}")

# Save results
results = {
    "metrics": {
        "severity_accuracy": severity_acc,
        "component_accuracy": component_acc,
        "reproducibility_accuracy": repro_acc,
        "overall_average": overall
    },
    "num_samples": len(test_data)
}

with open("/kaggle/working/evaluation_results.json", 'w') as f:
    json.dump(results, f, indent=2)

print("\n✅ Evaluation complete!")
print("Results saved to: /kaggle/working/evaluation_results.json")

## Step 4: Download Results

Download the trained model and evaluation results.

In [ ]:
# Zip the final model for download
!cd /kaggle/working && zip -r bug_classifier_model_v2.zip outputs/final_model/

print("\n✅ Model zipped!")
print("\nDownload these files:")
print("- /kaggle/working/bug_classifier_model_v2.zip")
print("- /kaggle/working/evaluation_results.json")
print("- /kaggle/working/train_improved.jsonl (optional - for analysis)")